## Proposal_Group7

## Introduction

### Background

Income inequality is an ongoing issue of great public concern in every country and in the United States it  has a history substantially older than the country itself (NPR,2016). Racial and ethnic communities in the United States are experiencing significant changes in rising income inequality (Kochhar.R & Cilluffo.A, 2018).There are notable wage gaps among American workers by race (Williams.W,2021). Based on these materials, this report aims to explore the correlation between salary and race which is one factor of income inequality. 

### Guiding Question

Is there a difference in means of salaries between people of Asian descent and people of White descent in the USA?

### Dataset

## Prelimary Results

In [7]:
library(cowplot)
library(datateachr)
library(digest)
library(infer)
library(repr)
library(taxyvr)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [8]:
salary_dataset <- read.csv("https://raw.githubusercontent.com/hongkailiu123/stat201_Group7/main/Salary_Data_Based_country_and_race.csv")

In [9]:
head(salary_dataset)

,X,Age,Gender,Education.Level,Job.Title,Years.of.Experience,Salary,Country,Race
,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,0,32,Male,Bachelor's,Software Engineer,5,90000,UK,White
2,1,28,Female,Master's,Data Analyst,3,65000,USA,Hispanic
3,2,45,Male,PhD,Senior Manager,15,150000,Canada,White
4,3,36,Female,Bachelor's,Sales Associate,7,60000,USA,Hispanic
5,4,52,Male,Master's,Director,20,200000,USA,Asian
6,5,29,Male,Bachelor's,Marketing Analyst,2,55000,USA,Hispanic


### Cleaning and Wrangling the Data

In [19]:
salary <- salary_dataset %>%
                filter(Country == "USA", 
                        Race == c("White", "Asian")) 

head(salary)

,X,Age,Gender,Education.Level,Job.Title,Years.of.Experience,Salary,Country,Race
,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,49,25,Male,Bachelor's,Help Desk Analyst,0,35000,USA,Asian
2,71,39,Female,Bachelor's,Recruiter,11,70000,USA,Asian
3,75,37,Male,Bachelor's,Project Manager,10,95000,USA,Asian
4,76,50,Female,Bachelor's,Operations Manager,22,160000,USA,White
5,107,36,Male,Bachelor's,IT Support Specialist,7,60000,USA,Asian
6,108,41,Female,Master's,Senior Marketing Analyst,14,100000,USA,White


## Methods: Plan

## References

Kochhar.R & Cilluffo.A, 2018. Income Inequality in the U.S is rising most rapidly among Asians\
https://www.pewresearch.org/social-trends/2018/07/12/income-inequality-in-the-u-s-is-rising-most-rapidly-among-asians/ \
NPR.(2016) Forgotten History: How the New England Colonists Embraced the Slave Trade\
https://www.npr.org/2016/06/21/482874478/forgotten-history-how-the-new-england-colonists-embraced-the-slave-trade \
Williams.W,2021. Wage gaps by race.\
https://www.investopedia.com/wage-gaps-by-race-5073258#citation-31 
